<a href="https://colab.research.google.com/gist/mahesh-ishran/9a35e7be6c43381719e1b590002932e5/indictts-deepfake-challenge-feature-extraction-lr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

indic_tts_deepfake_challenge_path = kagglehub.competition_download('indic-tts-deepfake-challenge')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current sessionfrom datasets import load_dataset

from datasets import load_dataset
import torch
import numpy as np
from transformers import Wav2Vec2FeatureExtractor, Wav2Vec2Model
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
import pandas as pd

# Check for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load dataset
dataset = load_dataset("SherryT997/IndicTTS-Deepfake-Challenge-Data")
train_data = dataset["train"]
test_data = dataset["test"]

# Use a subset of the dataset for training
train_data = train_data.select(range(20000))  # Select only first 10000 samples for training

# Initialize feature extractor
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("facebook/wav2vec2-base-960h")

def preprocess(batch):
    audio_arrays = [x["array"] for x in batch["audio"]]
    inputs = feature_extractor(audio_arrays, sampling_rate=16000, padding=True)
    return inputs

# Apply preprocessing
tiny_train_processed = train_data.map(
    preprocess,
    batched=True,
    batch_size=10,
    remove_columns=train_data.column_names
)

tiny_test_processed = test_data.map(
    preprocess,
    batched=True,
    batch_size=10,
    remove_columns=test_data.column_names
)

# Load model
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h").to(device)

def extract_embeddings(batch):
    inputs = torch.tensor(batch["input_values"]).to(device)
    with torch.no_grad():
        outputs = model(inputs)
    return {"embeddings": outputs.last_hidden_state.mean(dim=1).cpu().numpy()}

# Extract embeddings
tiny_train_embeddings = tiny_train_processed.map(
    extract_embeddings,
    batched=True,
    batch_size=10
)

tiny_test_embeddings = tiny_test_processed.map(
    extract_embeddings,
    batched=True,
    batch_size=10
)

# Save embeddings
np.save("tiny_train_embeddings.npy", tiny_train_embeddings["embeddings"])
np.save("tiny_labels.npy", train_data["is_tts"])
np.save("tiny_test_embeddings.npy", tiny_test_embeddings["embeddings"])
np.save("tiny_test_ids.npy", test_data["id"])

# Load embeddings
X_train = np.load("tiny_train_embeddings.npy")
y_train = np.load("tiny_labels.npy")
X_test = np.load("tiny_test_embeddings.npy")
test_ids = np.load("tiny_test_ids.npy")

# Train classifier
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)

# Predict on test set
y_test_pred = clf.predict_proba(X_test)[:, 1]

# Save submission
submission = pd.DataFrame({"id": test_ids, "is_tts": y_test_pred})
submission.to_csv("submission.csv", index=False)
print("Submission file saved as submission.csv")

